For this model I elected to stop using GridSearch and streamline my variables

In [30]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import patsy
import statsmodels.api as sm

from sklearn import metrics
from sklearn.metrics import r2_score, recall_score, make_scorer, f1_score, mean_squared_error

from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV

from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV , RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, VarianceThreshold, f_regression

from sklearn.preprocessing import StandardScaler, PolynomialFeatures #PowerTransformer

from sklearn.neighbors import KNeighborsRegressor


%config InlineBackend.figure_format = 'retina'
%matplotlib inline

pd.set_option('max_columns',300)

import warnings

### Model 4

In [31]:
X = pd.read_csv('../datasets/data_clean_poly.csv')

In [32]:
testset = pd.read_csv('../datasets/test_clean_poly.csv')

In [33]:
X.head()

,Id,PID,Lot Frontage,Lot Area,Alley,Lot Shape,Utilities,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating QC,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Val,Yr Sold,SalePrice,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,MS SubClass_60,MS SubClass_70,MS SubClass_75,MS SubClass_80,MS SubClass_85,MS SubClass_90,MS SubClass_120,MS SubClass_160,MS SubClass_180,MS SubClass_190,MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Pave,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,Neighborhood_dummies_Blueste,Neighborhood_dummies_BrDale,Neighborhood_dummies_BrkSide,Neighborhood_dummies_ClearCr,Neighborhood_dummies_CollgCr,Neighborhood_dummies_Crawfor,Neighborhood_dummies_Edwards,Neighborhood_dummies_Gilbert,Neighborhood_dummies_Greens,Neighborhood_dummies_IDOTRR,Neighborhood_dummies_MeadowV,Neighborhood_dummies_Mitchel,Neighborhood_dummies_NAmes,Neighborhood_dummies_NPkVill,Neighborhood_dummies_NWAmes,Neighborhood_dummies_NoRidge,Neighborhood_dummies_NridgHt,Neighborhood_dummies_OldTown,Neighborhood_dummies_SWISU,Neighborhood_dummies_Sawyer,Neighborhood_dummies_SawyerW,Neighborhood_dummies_Somerst,Neighborhood_dummies_StoneBr,Neighborhood_dummies_Timber,Neighborhood_dummies_Veenker,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Plywood,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasW,Heating_Grav,Heating_OthW,Central Air_Y,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_WD,Season_Spring,Season_Summer,Season_Winter,Mo Sold_2,Mo Sold_3,Mo Sold_4,Mo Sold_5,Mo Sold_6,Mo Sold_7,Mo Sold_8,Mo Sold_9,Mo Sold_10,Mo Sold_11,Mo Sold_12,Exter Qual 2,Exter Cond 2,Kitchen Qual 2,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_None,Alley Grvl,Alley Pave,Misc Feature_Gar2,Misc Feature_Othr,Misc Feature_Shed,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_Stone,Overall Qual Gr Liv Area,Overall Qual^2,Overall Qual Garage Cars,Overall Qual Bsmt Qual,Overall Qual Garage Area,Bsmt Qual Gr Liv Area,Overall Qual Year Built,Overall Qual Garage Yr Blt,Overall Qual Year Remod/Add,Overall Qual Yr Sold,Overall Qual TotRms AbvGrd,Gr Liv Area Garage Cars,Overall Qual 1st Flr SF,Overall Qual Street_Pave,Garage Cars Kitchen Qual 2,Garage Area Kitchen Qual 2,Garage Cars Exter Qual 2,Gr Liv

In [34]:
testset.head()

,Id,PID,Lot Frontage,Lot Area,Alley,Lot Shape,Utilities,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating QC,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Val,Yr Sold,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,MS SubClass_60,MS SubClass_70,MS SubClass_75,MS SubClass_80,MS SubClass_85,MS SubClass_90,MS SubClass_120,MS SubClass_160,MS SubClass_180,MS SubClass_190,MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Street_Pave,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Bldg Type_2fmCon,Bldg Type_Duplex,Bldg Type_Twnhs,Bldg Type_TwnhsE,Neighborhood_dummies_Blueste,Neighborhood_dummies_BrDale,Neighborhood_dummies_BrkSide,Neighborhood_dummies_ClearCr,Neighborhood_dummies_CollgCr,Neighborhood_dummies_Crawfor,Neighborhood_dummies_Edwards,Neighborhood_dummies_Gilbert,Neighborhood_dummies_Greens,Neighborhood_dummies_IDOTRR,Neighborhood_dummies_MeadowV,Neighborhood_dummies_Mitchel,Neighborhood_dummies_NAmes,Neighborhood_dummies_NPkVill,Neighborhood_dummies_NWAmes,Neighborhood_dummies_NoRidge,Neighborhood_dummies_NridgHt,Neighborhood_dummies_OldTown,Neighborhood_dummies_SWISU,Neighborhood_dummies_Sawyer,Neighborhood_dummies_SawyerW,Neighborhood_dummies_Somerst,Neighborhood_dummies_StoneBr,Neighborhood_dummies_Timber,Neighborhood_dummies_Veenker,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_MetalSd,Exterior 1st_Plywood,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsphShn,Exterior 2nd_Brk Cmn,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CmentBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Plywood,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_Wd Shng,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Central Air_Y,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_WD,Season_Spring,Season_Summer,Season_Winter,Mo Sold_2,Mo Sold_3,Mo Sold_4,Mo Sold_5,Mo Sold_6,Mo Sold_7,Mo Sold_8,Mo Sold_9,Mo Sold_10,Mo Sold_11,Mo Sold_12,Exter Qual 2,Exter Cond 2,Kitchen Qual 2,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_None,Alley Grvl,Alley Pave,Misc Feature_Gar2,Misc Feature_Othr,Misc Feature_Shed,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_Stone,Overall Qual Gr Liv Area,Overall Qual^2,Overall Qual Garage Cars,Overall Qual Bsmt Qual,Overall Qual Garage Area,Bsmt Qual Gr Liv Area,Overall Qual Year Built,Overall Qual Garage Yr Blt,Overall Qual Year Remod/Add,Overall Qual Yr Sold,Overall Qual TotRms AbvGrd,Gr Liv Area Garage Cars,Overall Qual 1st Flr SF,Overall Qual Street_Pave,Garage Cars Kitchen Qual 2,Garage Area Kitchen Qual 2,Garage Cars Exter Qual 2,Gr Liv Area Kitchen Q

In [35]:
features = list(abs(X.corrwith(X['SalePrice']).sort_values(ascending=False))[0:100].index)

In [36]:
X = X[features]

In [37]:
y = X['SalePrice']
X.drop('SalePrice', axis=1, inplace=True)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

param_grid_1 = {
    'alpha':[.1,.3,.5,.6,.7,.8,1,1.5],
    'l1_ratio':[0,.3,.5,.7,1]
}

gs_simple = GridSearchCV(ElasticNet(),param_grid_1,cv=5,verbose=1)

gs_simple.fit(X_train_sc,y_train_log)  

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increa

GridSearchCV(cv=5, error_score='raise',
       estimator=ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.1, 0.3, 0.5, 0.6, 0.7, 0.8, 1, 1.5], 'l1_ratio': [0, 0.3, 0.5, 0.7, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [17]:
gs_simple.best_score_

0.8723271592552025

In [40]:
## 2000 by 99
X_SC = ss.fit_transform(X)

In [27]:
gs_en_log = gs_simple.best_estimator_



0.8855697918813392

In [42]:
features.remove('SalePrice')

In [43]:
testset = testset[features]

In [44]:
testset.shape

(879, 99)

In [49]:
X.head()

,Overall Qual Gr Liv Area,Overall Qual^2,Overall Qual Garage Cars,Overall Qual Bsmt Qual,Overall Qual Garage Area,Bsmt Qual Gr Liv Area,Overall Qual Year Built,Overall Qual Garage Yr Blt,Overall Qual Year Remod/Add,Overall Qual,Overall Qual Yr Sold,Overall Qual TotRms AbvGrd,Gr Liv Area Garage Cars,Overall Qual 1st Flr SF,Overall Qual Street_Pave,Garage Cars Kitchen Qual 2,Garage Area Kitchen Qual 2,Garage Cars Exter Qual 2,Gr Liv Area Kitchen Qual 2,Garage Area Exter Qual 2,Gr Liv Area Exter Qual 2,Bsmt Qual Garage Area,Bsmt Qual Garage Cars,Overall Qual Total Bsmt SF,Total Bsmt SF Garage Cars,Overall Qual Kitchen Qual 2,Overall Qual Exter Qual 2,1st Flr SF Garage Cars,TotRms AbvGrd Exter Qual 2,1st Flr SF Kitchen Qual 2,Overall Qual Full Bath,Gr Liv Area Garage Area,1st Flr SF Exter Qual 2,Total Bsmt SF Kitchen Qual 2,Bsmt Qual 1st Flr SF,Bsmt Qual Exter Qual 2,TotRms AbvGrd Kitchen Qual 2,Bsmt Qual Kitchen Qual 2,Kitchen Qual Garage Cars,Full Bath Kitchen Qual 2,Gr Liv Area Garage Finish,Garage Finish Kitchen Qual 2,Total Bsmt SF Exter Qual 2,Garage Finish Garage Area,Exter Qual 2 Kitchen Qual 2,Kitchen Qual Garage Area,Garage Finish Exter Qual 2,Full Bath Garage Area,Fireplace Qu Exter Qual 2,Exter Qual 2,Exter Qual,Gr Liv Area,Kitchen Qual,Kitchen Qual 2,Garage Area,Garage Cars,Total Bsmt SF,1st Flr SF,Bsmt Qual,Year Built,Garage Finish,Garage Yr Blt,Year Remod/Add,Full Bath,Foundation_PConc,Fireplace Qu,Mas Vnr Area,TotRms AbvGrd,Fireplaces,Neighborhood_dummies_NridgHt,Heating QC,BsmtFin SF 1,Bsmt Exposure,Sale Type_New,BsmtFin Type 1,Garage Type_Attchd,MS SubClass_60,Exterior 1st_VinylSd,Exterior 2nd_VinylSd,Lot Frontage,Open Porch SF,Wood Deck SF,Mas Vnr Type_Stone,Lot Area,Lot Shape,Paved Drive,Bsmt Full Bath,Half Bath,Central Air_Y,Garage Qual,Roof Style_Hip,Neighborhood_dummies_NoRidge,Mas Vnr Type_BrkFace,Neighborhood_dummies_StoneBr,Garage Cond,2nd Flr SF,MS Zoning_RL,Garage Type_BuiltIn,Land Contour_HLS
0,8874,36,12,15.0,2850,3697.5,11856,11856,12030,6,12060,36,2958,4350,6,2,475,2,1479,475,1479,1187.5,5.0,4350,1450,6,6,1450,6,725,12,702525,725,725,1812.5,2.5,6,2.5,2,2,2958,2,725,950,1,475,2,950,0.0,1,1,1479,1,1,475,2,725,725,2.5,1976,2,1976,2005,2,0,0.0,289,6,0,0,2,533,0,0,4.0,1,1,0,0,72.774648,44,0,0,13517,1,2,0,1,1,2.5,0,0,1,0,2.5,754,1,0,0
1,14854,49,14,21.0,3913,6366.0,13972,13979,13979,7,14063,56,4244,6391,7,2,559,2,2122,559,2122,1677.0,6.0,6391,1826,7,7,1826,8,913,14,1186198,913,913,2739.0,3.0,8,3.0,2,2,4244,2,913,1118,1,559,2,1118,2.5,1,1,2122,1,1,559,2,913,913,3.0,1996,2,1997,1997,2,1,2.5,132,8,1,0,2,637,0,0,4.0,1,1,1,1,43.000000,74,0,0,11492,1,2,1,1,1,2.5,0,0,1,0,2.5,1209,1,0,0
2,5285,25,5,12.5,1230,2642.5,9765,9765,10035,5,10050,25,1057,5285,5,1,246,0,1057,0,0,615.0,2.5,5285,1057,5,0,1057,0,1057,5,260022,0,1057,2642.5,0.0,5,2.5,1,1,1057,1,0,246,0,246,0,246,0.0,0,0,1057,1,1,246,1,1057,1057,2.5,1953,1,1953,2007,1,0,0.0,0,5,0,0,0,731,0,0,4.0,0,0,1,1,68.000000,52,0,0,7922,0,2,1,0,1,2.5,0,0,0,0,2.5,0,1,0,0
3,7220,25,10,15.0,2000,4332.0,10030,10035,10035,5,10050,35,2888,3720,5,0,0,0,0,0,0,1200.0,6.0,1920,768,0,0,1488,0,0,10,577600,0,0,2232.0,0.0,0,0.0,0,0,4332,0,0,1200,0,0,0,800,0.0,0,0,1444,0,0,400,2,384,744,3.0,2006,3,2007,2007,2,1,0.0,0,7,0,0,1,0,0,0,1.5,0,1,1,1,73.000000,0,100,0,9802,0,2,0,1,1,2.5,0,0,0,0,2.5,700,1,1,0
4,8670,36,12,12.0,2904,2890.0,11400,11742,11958,6,12060,36,2890,4986,6,0,0,0,0,0,0,968.0,4.0,4056,1352,0,0,1662,0,0,12,699380,0,0,1662.0,0.0,0,0.0,0,0,1445,0,0,484,0,0,0,968,0.0,0,0,1445,0,0,484,2,676,831,2.0,1900,1,1957,1993,2,1,0.0,0,6,0,0,0,0,0,0,1.5,0,0,0,0,82.000000,59,0,0,14235,1,0,0,0,1,2.5,0,0,0,0,2.5,614,1,0,0


In [23]:
testset.shape

(879, 99)

In [24]:
testset.head()

,Overall Qual Gr Liv Area,Overall Qual^2,Overall Qual Garage Cars,Overall Qual Bsmt Qual,Overall Qual Garage Area,Bsmt Qual Gr Liv Area,Overall Qual Year Built,Overall Qual Garage Yr Blt,Overall Qual Year Remod/Add,Overall Qual,Overall Qual Yr Sold,Overall Qual TotRms AbvGrd,Gr Liv Area Garage Cars,Overall Qual 1st Flr SF,Overall Qual Street_Pave,Garage Cars Kitchen Qual 2,Garage Area Kitchen Qual 2,Garage Cars Exter Qual 2,Gr Liv Area Kitchen Qual 2,Garage Area Exter Qual 2,Gr Liv Area Exter Qual 2,Bsmt Qual Garage Area,Bsmt Qual Garage Cars,Overall Qual Total Bsmt SF,Total Bsmt SF Garage Cars,Overall Qual Kitchen Qual 2,Overall Qual Exter Qual 2,1st Flr SF Garage Cars,TotRms AbvGrd Exter Qual 2,1st Flr SF Kitchen Qual 2,Overall Qual Full Bath,Gr Liv Area Garage Area,1st Flr SF Exter Qual 2,Total Bsmt SF Kitchen Qual 2,Bsmt Qual 1st Flr SF,Bsmt Qual Exter Qual 2,TotRms AbvGrd Kitchen Qual 2,Bsmt Qual Kitchen Qual 2,Kitchen Qual Garage Cars,Full Bath Kitchen Qual 2,Gr Liv Area Garage Finish,Garage Finish Kitchen Qual 2,Total Bsmt SF Exter Qual 2,Garage Finish Garage Area,Exter Qual 2 Kitchen Qual 2,Kitchen Qual Garage Area,Garage Finish Exter Qual 2,Full Bath Garage Area,Fireplace Qu Exter Qual 2,Exter Qual 2,Exter Qual,Gr Liv Area,Kitchen Qual,Kitchen Qual 2,Garage Area,Garage Cars,Total Bsmt SF,1st Flr SF,Bsmt Qual,Year Built,Garage Finish,Garage Yr Blt,Year Remod/Add,Full Bath,Foundation_PConc,Fireplace Qu,Mas Vnr Area,TotRms AbvGrd,Fireplaces,Neighborhood_dummies_NridgHt,Heating QC,BsmtFin SF 1,Bsmt Exposure,Sale Type_New,BsmtFin Type 1,Garage Type_Attchd,MS SubClass_60,Exterior 1st_VinylSd,Exterior 2nd_VinylSd,Lot Frontage,Open Porch SF,Wood Deck SF,Mas Vnr Type_Stone,Lot Area,Lot Shape,Paved Drive,Bsmt Full Bath,Half Bath,Central Air_Y,Garage Qual,Roof Style_Hip,Neighborhood_dummies_NoRidge,Mas Vnr Type_BrkFace,Neighborhood_dummies_StoneBr,Garage Cond,2nd Flr SF,MS Zoning_RL,Garage Type_BuiltIn,Land Contour_HLS
0,8874,36,12,15.0,2850,3697.5,11856,11856,12030,6,12060,36,2958,4350,6,2,475,2,1479,475,1479,1187.5,5.0,4350,1450,6,6,1450,6,725,12,702525,725,725,1812.5,2.5,6,2.5,2,2,2958,2,725,950,1,475,2,950,0.0,0,0,1928,-1,-1,440,1,1020,908,2.0,1910,1,1910,1950,2,0,0.0,0,9,0,0,1,0,0,0,1.5,0,0,0,0,69.000000,60,0,0,9142,0,2,0,0,0,1.5,0,0,0,0,1.5,1020,0,0,0
1,14854,49,14,21.0,3913,6366.0,13972,13979,13979,7,14063,56,4244,6391,7,2,559,2,2122,559,2122,1677.0,6.0,6391,1826,7,7,1826,8,913,14,1186198,913,913,2739.0,3.0,8,3.0,2,2,4244,2,913,1118,1,559,2,1118,2.5,0,0,1967,0,0,580,2,1967,1967,3.0,1977,3,1977,1977,2,0,0.0,0,10,0,0,0,0,0,0,1.5,1,0,0,0,79.222222,0,170,0,9662,1,2,0,0,1,2.5,0,0,0,0,2.5,0,1,0,0
2,5285,25,5,12.5,1230,2642.5,9765,9765,10035,5,10050,25,1057,5285,5,1,246,0,1057,0,0,615.0,2.5,5285,1057,5,0,1057,0,1057,5,260022,0,1057,2642.5,0.0,5,2.5,1,1,1057,1,0,246,0,246,0,246,0.0,1,1,1496,1,1,426,2,654,664,3.0,2006,2,2006,2006,2,1,3.0,0,7,1,0,2,554,2,1,4.0,1,1,1,1,58.000000,24,100,0,17104,1,2,1,1,1,2.5,0,0,0,0,2.5,832,1,0,0
3,7220,25,10,15.0,2000,4332.0,10030,10035,10035,5,10050,35,2888,3720,5,0,0,0,0,0,0,1200.0,6.0,1920,768,0,0,1488,0,0,10,577600,0,0,2232.0,0.0,0,0.0,0,0,4332,0,0,1200,0,0,0,800,0.0,1,1,968,0,0,480,2,968,968,2.5,1923,1,1935,2006,1,0,0.0,0,5,0,0,0,0,0,0,1.5,0,0,0,0,60.000000,0,0,0,8520,0,0,0,0,1,2.0,0,0,0,0,2.5,0,0,0,0
4,8670,36,12,12.0,2904,2890.0,11400,11742,11958,6,12060,36,2890,4986,6,0,0,0,0,0,0,968.0,4.0,4056,1352,0,0,1662,0,0,12,699380,0,0,1662.0,0.0,0,0.0,0,0,1445,0,0,484,0,0,0,968,0.0,0,0,1394,0,0,514,2,1394,1394,3.0,1963,2,1963,1963,1,0,3.0,247,6,2,0,1,609,0,0,3.0,1,0,0,0,75.254545,76,0,0,9500,1,2,1,1,1,2.5,0,0,1,0,2.5,0,1,0,0


C:\Users\eamon\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [ ]:
testset_sc = ss.transform(testset)

y_log = np.log(y)

gs_en_log.fit(X_SC,y_log)  

predictions = gs_en_log.predict(testset_sc)

In [46]:
predictions = np.exp(predictions)
predictions

array([120897.09853329, 191304.19053632, 191669.46024406, 109717.7963824 ,
       170043.35493688, 107387.74116251, 126840.56619705, 144121.16430978,
       144950.10105917, 149088.90454059, 165231.67645998, 130106.13676535,
       133188.13709038, 298319.51170795, 150528.52211044, 143036.35305184,
       136206.78103022, 125349.99740409, 191077.37951917, 193680.5670976 ,
       157377.23019277, 153492.52841674, 222563.13676936, 171335.70217754,
       156630.8455731 , 131938.06091604, 151870.12228603, 121138.55616091,
       146453.0724425 ,  74811.39359913, 123726.09354917, 135708.55893597,
       178221.69810149, 165345.53801195, 241588.12088465, 159837.70547529,
       107168.21638787,  97171.35625338, 151682.38059745, 151557.15575366,
       132843.53984904, 241652.16914935, 148170.50557505, 168737.00871163,
       188354.08004443, 137919.09561296, 195802.86029433, 137149.99234365,
       127465.45097256, 143953.03181092, 120723.90516446, 138083.95633302,
       224097.1540613 , 1

In [55]:
X_file = pd.read_csv('../datasets/test.csv')

In [56]:
testset['Id'] = X_file['Id']

In [57]:
testset['SalePrice'] = predictions
submission = testset[['Id','SalePrice']]
submission

,Id,SalePrice
0,2658,120897.098533
1,2718,191304.190536
2,2414,191669.460244
3,1989,109717.796382
4,625,170043.354937
5,333,107387.741163
6,1327,126840.566197
7,858,144121.164310
8,95,144950.101059
9,1568,149088.904541


In [58]:
submission[submission['Id']==2]

,Id,SalePrice
704,2,101543.471914


In [60]:
submission.to_csv('../datasets/ef_ames_12_6_E.csv',index=False)